https://documenter.getpostman.com/view/13485071/2s93sgXWUY?utm_campaign=API&utm_source=gs&utm_medium=email&utm_content=sign-up#ea172e5b-8467-4c95-b5e9-83288ee94e24

- Grab rating breakdown over demographic

### Desired Features for each zip code
- Average composite rating by type of school (and public v private?)
- [OPTIONAL] Themed ratings: test scores, college readiness, and equity scores
- [OPTIONAL] student-teacher ratio, enrollment count

In [63]:
import sys

# !{sys.executable} -m pip install joblib
# !{sys.executable} -m pip install geopy

You are using pip version 19.0.3, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [132]:
import pandas as pd
import requests
import configparser
import os
import datetime as dt
import numpy as np
from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
# from geopy.geocoders import Nominatim

In [7]:
def get_lat_lon_from_zip(zip_code, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address=USA+{zip_code:05d}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']['location']
            return (location['lat'], location['lng'])
        else:
            return None
    else:
        return None

In [8]:
# Get Great Schools API Credentials
config = configparser.ConfigParser()
config.read('../SECRETS.ini')
GS_API_KEY = config['great_schools_api']['key']
GG_API_KEY = config['google_geocode_api']['key']

gs_call_count = int(config['great_schools_api']['call_count'])
gs_call_count

24464

### Explore

In [9]:
zip_code = 49871

lat, lon = get_lat_lon_from_zip(zip_code, GG_API_KEY)
lat, lon

(46.4410908, -87.59292459999999)

In [10]:
url = f'https://gs-api.greatschools.org/schools?zip={zip_code}'
payload={}
headers = {
  'Accept': 'application/json',
  'Content': 'application/json',
  'X-API-Key': GS_API_KEY
}

response_zip = requests.request("GET", url, headers=headers, data=payload)

gs_call_count += 1

config['great_schools_api']['call_count'] = str(gs_call_count)
with open('../SECRETS.ini', 'w') as file:
    config.write(file)

response_zip.json()

{'schools': [],
 'cur_page': 0,
 'items_per_page': 50,
 'max_page_num': 0,
 'total_count': 0,
 'links': {'self': '/schools?zip=49871&page=0',
  'prev': '',
  'next': '',
  'first': '/schools?zip=49871&page=0',
  'last': '/schools?zip=49871&page=0'}}

In [98]:
url = f'https://gs-api.greatschools.org/nearby-schools?lat={lat}&lon={lon}&limit=50&distance=20'
payload={}
headers = {
  'Accept': 'application/json',
  'Content': 'application/json',
  'X-API-Key': GS_API_KEY
}

response_dist = requests.request("GET", url, headers=headers, data=payload)

gs_call_count += 1

config['great_schools_api']['call_count'] = str(gs_call_count)
with open('../SECRETS.ini', 'w') as file:
    config.write(file)

response_dist.json()

{'schools': [{'universal-id': '2608745',
   'nces-id': '261953002051',
   'state-id': '08864',
   'name': 'Ishpeming-Negaunee-Nice Community Ed. Division',
   'school-summary': 'Ishpeming-Negaunee-Nice Community Ed. Division, a public school located in Negaunee, MI, serves grades 7-12 in the Ishpeming Public School District No. 1.It has received a GreatSchools Rating of 5 out of 10, based on a variety of school quality measures.',
   'type': 'public',
   'level-codes': 'm,h',
   'level': '7,8,9,10,11,12',
   'street': '101 South Pioneer Avenue',
   'city': 'Negaunee',
   'state': 'MI',
   'fipscounty': 26103,
   'zip': '49866',
   'phone': '(906) 475-4173',
   'fax': '(906) 475-7443',
   'county': 'Marquette County',
   'lat': 46.499416,
   'lon': -87.608871,
   'district-name': 'Ishpeming Public School District No. 1',
   'district-id': 461,
   'web-site': None,
   'overview-url': 'https://www.greatschools.org/michigan/negaunee/8745-Ishpeming-Negaunee-Nice-Community-Ed.-Division/',
  

In [11]:
pd.DataFrame(response_dist.json()['schools'])

,universal-id,nces-id,state-id,name,school-summary,type,level-codes,level,street,city,...,lat,lon,district-name,district-id,web-site,overview-url,rating,year,rating-description,distance
0,0633620,A1900720,None,Sofos Preparatory Academy,"Sofos Preparatory Academy, a private school lo...",private,"m,h","8,9,10,11",1009 E CAPITOL EXPY # 610,San Jose,...,37.302109,-121.805016,None,0,None,https://www.greatschools.org/california/san-jo...,None,NaN,The GreatSchools Rating helps parents compare ...,0.000208
1,0610273,A9500584,43694357008956,Scholars Academy,"Scholars Academy, a private school located in ...",private,e,"KG,1,2,3",3703 Silver Creek Road,San Jose,...,37.301971,-121.806412,None,0,http://scholarsacademyschool.com/,https://www.greatschools.org/california/san-jo...,None,NaN,The GreatSchools Rating helps parents compare ...,0.077498
2,0605465,061182001309,43694274337903,Silver Creek High School,"Silver Creek High School, a public school loca...",public,h,"9,10,11,12",3434 Silver Creek Road,San Jose,...,37.303486,-121.806572,East Side Union High School District,632,http://silvercreek.esuhsd.org/,https://www.greatschools.org/california/san-jo...,8,2024.0,The GreatSchools Rating helps parents compare ...,0.128106
3,0626335,None,43694356140750,Alim Academy,"Alim Academy, a private school located in San ...",private,"e,m,h","1,2,3,4,5,6,7,8,9,10,11,12",3924 Picardy Place Court,San Jose,...,37.297325,-121.804337,None,0,None,https://www.greatschools.org/california/san-jo...,None,NaN,The GreatSchools Rating helps parents compare ...,0.332555
4,0605477,061314001495,43694356095988,John J. Montgomery Elementary School,"John J. Montgomery Elementary School, a public...",public,e,"KG,1,2,3,4,5,6",2010 Daniel Maloney Drive,San Jose,...,37.306591,-121.801094,Evergreen Elementary School District,633,http://montgomery.eesd.org/,https://www.greatschools.org/california/san-jo...,4,2024.0,The GreatSchools Rating helps parents compare ...,0.377252
5,0614076,061437011452,43694500108696,Ramblewood Elementary School,"Ramblewood Elementary School, a public school ...",public,e,"KG,1,2,3,4,5,6",1351 Lightland Road,San Jose,...,37.296051,-121.812340,Franklin-Mckinley Elementary School District,634,http://ramblewood.fmsd.org/,https://www.greatschools.org/california/san-jo...,5,2024.0,The GreatSchools Rating helps parents compare ...,0.580813
6,0605474,061314001493,43694356085690,George V. Leyva Intermediate School,"George V. Leyva Intermediate School, a public ...",public,m,"7,8",1865 Monrovia Drive,San Jose,...,37.314327,-121.812248,Evergreen Elementary School District,633,None,https://www.greatschools.org/california/san-jo...,5,2024.0,The GreatSchools Rating helps parents compare ...,0.933208
7,0605467,061314001490,43694356047120,Cadwallader Elementary School,"Cadwallader Elementary School, a public school...",public,e,"KG,1,2,3,4,5,6",3799 Cadwallader Avenue,San Jose,...,37.306580,-121.788788,Evergreen Elementary School District,633,http://www.do.esd.k12.ca.us/Cadwallader/cad.html,https://www.greatschools.org/california/san-jo...,8,2024.0,The GreatSchools Rating helps parents compare ...,0.943697
8,0611864,061314008631,43694356117956,James Franklin Smith Elementary School,"James Franklin Smith Elementary School, a publ...",public,e,"KG,1,2,3,4,5,6",2220 Woodbury Lane,San Jose,...,37.297146,-121.787125,Evergreen Elementary School District,633,http://evergreen.eesd.org/page.cfm?p=3666,https://www.greatschools.org/california/san-jo...,8,2024.0,The GreatSchools Rating helps parents compare ...,1.041200
9,0608798,00087751,43694356940522,Liberty Baptist School,"Liberty Baptist School, a private school locat...",private,"p,e,m,h","PK,TK,KG,1,2,3,4,5,6,7,8,9,10,11,12",2790 South King Road,San Jose,...,37.314545,-121.816895,None,0,None,https://www.greatschools.org/california/san-jo...,None,NaN,The GreatSchools Rating helps parents compare ...,1.079285


In [14]:
df = pd.DataFrame(response_dist.json()['schools'])
df['query_zip_code'] = zip_code
# df.dropna(subset=['rating'])

## Main Extract

In [99]:
print(f'Calls remaining: {200000 - gs_call_count}')

all_zips = pd.read_csv('../data/raw/all_zip_codes.csv')
gs_ratings_path = '../data/raw/great_schools_ratings.csv'
if os.path.exists(gs_ratings_path):
    logged_zips = list(pd.read_csv(gs_ratings_path)['query_zip_code'].unique())
else:
    logged_zips = []
    
dfs = []
num_zips = all_zips.shape[0]
i = 0
for zc in all_zips['0']:
    if zc in logged_zips:
        continue
    else:
        try:
            lat, lon = get_lat_lon_from_zip(zc, GG_API_KEY)
        except Exception as e:
            print(f'No lat/lon coordinates avaiable for zip code: {zc}\nSkipping...')
            continue
        log_coords = (zc, lat, lon)
        url = f'https://gs-api.greatschools.org/nearby-schools?lat={lat}&lon={lon}&limit=50&distance=20'
        payload={}
        headers = {
          'Accept': 'application/json',
          'Content': 'application/json',
          'X-API-Key': GS_API_KEY
        }
        
        try:
            response_dist = requests.request("GET", url, headers=headers, data=payload)
            gs_call_count += 1
            df = pd.DataFrame(response_dist.json()['schools'])
            df['query_zip_code'] = zc
            df['cache_date'] = dt.datetime.now().date()
            dfs.append(df)
        except Exception as e:
            print(f'Zip Code Failed: {zc}\nException: {e}\n\nExiting and caching...')
            break
        
        if (len(logged_zips) + i) % 100 == 0:
            print(f'Zip Codes logged = {len(logged_zips) + i}/{num_zips}')
        i+=1

if os.path.exists(gs_ratings_path):
    df_gs_logged = pd.read_csv(gs_ratings_path)
    df_gs = pd.concat([df_gs_logged]+dfs)
    df_gs.to_csv(gs_ratings_path, index=False)
else:
    df_gs = pd.concat(dfs)  
    df_gs.to_csv(gs_ratings_path, index=False)
    
config['great_schools_api']['call_count'] = str(gs_call_count)
with open('../SECRETS.ini', 'w') as file:
    config.write(file)  

Calls remaining: 180454
No lat/lon coordinates avaiable for zip code: 19542
Skipping...
No lat/lon coordinates avaiable for zip code: 85025
Skipping...
Zip Codes logged = 19400/24223
Zip Codes logged = 19500/24223
Zip Codes logged = 19600/24223
Zip Codes logged = 19700/24223
Zip Codes logged = 19800/24223
Zip Codes logged = 19900/24223
Zip Codes logged = 20000/24223
Zip Codes logged = 20100/24223
Zip Codes logged = 20200/24223
Zip Codes logged = 20300/24223
Zip Codes logged = 20400/24223
Zip Codes logged = 20500/24223
Zip Codes logged = 20600/24223
Zip Codes logged = 20700/24223
Zip Codes logged = 20800/24223
Zip Codes logged = 20900/24223
Zip Codes logged = 21000/24223
Zip Codes logged = 21100/24223
Zip Codes logged = 21200/24223
Zip Codes logged = 21300/24223
Zip Codes logged = 21400/24223
Zip Codes logged = 21500/24223
Zip Codes logged = 21600/24223
Zip Codes logged = 21700/24223
Zip Codes logged = 21800/24223
Zip Codes logged = 21900/24223
Zip Codes logged = 22000/24223
Zip Codes l

In [101]:
df_gs.head()

,universal-id,nces-id,state-id,name,school-summary,type,level-codes,level,street,city,state,fipscounty,zip,phone,fax,county,lat,lon,district-name,district-id,web-site,overview-url,rating,year,rating-description,distance,query_zip_code,cache_date
0,3605488,A9104735,430300999233,Calvary Chapel Academy,"Calvary Chapel Academy, a private school locat...",private,"e,m","KG,1,2,3,4,5,6,7",1777 New York 332,Farmington,NY,36069.0,14425,(585) 398-2218,NaN,Ontario County,42.946701,-77.334038,NaN,0.0,NaN,https://www.greatschools.org/new-york/farmingt...,NaN,NaN,The GreatSchools Rating helps parents compare ...,2.492651,14425,2024-06-28
1,3603930,362964003993,431701060001,Victor Intermediate School,"Victor Intermediate School, a public school lo...",public,e,"4,5,6,UG",953 High Street,Victor,NY,36069.0,14564,(585) 924-3252,NaN,Ontario County,42.989552,-77.414680,Victor Central School District,647.0,http://www.victorschools.org,https://www.greatschools.org/new-york/victor/3...,5.0,2024.0,The GreatSchools Rating helps parents compare ...,3.613472,14425,2024-06-28
2,3603931,362964003994,431701060002,Victor Junior High School,"Victor Junior High School, a public school loc...",public,m,"7,8",953 High Street,Victor,NY,36069.0,14564,(585) 924-3252,(585) 924-9535,Ontario County,42.989552,-77.414680,Victor Central School District,647.0,http://www.victorschools.org,https://www.greatschools.org/new-york/victor/3...,7.0,2024.0,The GreatSchools Rating helps parents compare ...,3.613472,14425,2024-06-28
3,3603932,362964003995,431701060003,Victor Primary School,"Victor Primary School, a public school located...",public,e,"KG,1,2,3,UG",953 High Street,Victor,NY,36069.0,14564,(585) 924-3252,NaN,Ontario County,42.989552,-77.414680,Victor Central School District,647.0,http://www.victorschools.org,https://www.greatschools.org/new-york/victor/3...,3.0,2024.0,The GreatSchools Rating helps parents compare ...,3.613472,14425,2024-06-28
4,3603933,362964003996,431701060004,Victor Senior High School,"Victor Senior High School, a public school loc...",public,h,"9,10,11,12,UG",953 High Street,Victor,NY,36069.0,14564,(585) 924-3252,NaN,Ontario County,42.989552,-77.414680,Victor Central School District,647.0,http://www.victorschools.org,https://www.greatschools.org/new-york/victor/3...,7.0,2024.0,The GreatSchools Rating helps parents compare ...,3.613472,14425,2024-06-28


## Group Data
Create a final dataset that shows the average rating for each zip code by school type and level

In [16]:
gs_ratings_path = '../data/raw/great_schools_ratings.csv'
df_gs = pd.read_csv(gs_ratings_path)
df_gs1 = df_gs.copy()[['query_zip_code','distance','type','level-codes','rating']]
df_gs1.dropna(subset =['rating'], inplace=True)
df_gs1.sort_values(['query_zip_code','distance'], ascending=True)
# df_gs1 = df_gs1.groupby(['query_zip_code',])
df_gs1.rating = pd.to_numeric(df_gs1.rating)
# df_gs1['pre_k'] = 0
# df_gs1['elementary'] = 0
# df_gs1['middle'] = 0
# df_gs1['high'] = 0
level_families = {
    'p':'Pre-K',
    'e':'Elementary School',
    'm':'Middle School',
    'h':'High School'
}

df_gs1['level_family'] = np.nan

dfs = []
for code, family in level_families.items():
    df_tmp = df_gs1.loc[df_gs1['level-codes'].str.contains(code)]
    df_tmp['level_family'] = family
    dfs.append(df_tmp)

df_gs2 = pd.concat(dfs).groupby(['query_zip_code','type','level_family']).head(3)

dfs = []
for st in list(df_gs2.type.unique()):
    for lf in list(df_gs2.level_family.unique()):
        df_tmp = df_gs2.loc[
            (df_gs2.type == st) &
            (df_gs2.level_family == lf)
        ].groupby(['query_zip_code','type','level_family']).mean()
        dfs.append(df_tmp)
        
df_gs3 = pd.concat(dfs).reset_index().sort_values(['query_zip_code','type']).rename(
    columns={'query_zip_code':'zip_code'})
# df_gs3.to_csv('../data/processed/great_schools_mean_ratings.csv', index=False)


C:\Users\wheat\OneDrive\Dev\project_zipliner\zipliner\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [61]:
sorted(df_gs.dropna(subset=['rating']).sort_values(['query_zip_code','distance'])\
.groupby(['query_zip_code','type']).head(12)['universal-id'].unique())

[100005,
 100006,
 100007,
 100009,
 100010,
 100012,
 100013,
 100016,
 100017,
 100018,
 100019,
 100021,
 100024,
 100026,
 100027,
 100028,
 100029,
 100030,
 100031,
 100032,
 100033,
 100034,
 100035,
 100036,
 100037,
 100038,
 100039,
 100040,
 100041,
 100042,
 100043,
 100049,
 100050,
 100051,
 100052,
 100053,
 100060,
 100062,
 100064,
 100068,
 100069,
 100070,
 100071,
 100072,
 100073,
 100074,
 100075,
 100076,
 100077,
 100078,
 100079,
 100080,
 100082,
 100083,
 100084,
 100086,
 100087,
 100088,
 100089,
 100090,
 100091,
 100092,
 100093,
 100094,
 100098,
 100099,
 100100,
 100101,
 100102,
 100103,
 100104,
 100105,
 100106,
 100109,
 100112,
 100113,
 100114,
 100115,
 100116,
 100117,
 100118,
 100119,
 100120,
 100121,
 100122,
 100123,
 100124,
 100125,
 100126,
 100127,
 100128,
 100131,
 100134,
 100136,
 100137,
 100141,
 100143,
 100144,
 100145,
 100146,
 100147,
 100148,
 100149,
 100150,
 100151,
 100152,
 100153,
 100154,
 100155,
 100157,
 100162,
 

In [137]:
df_gs3.sort_values(['zip_code','type']).head(25)

,zip_code,type,level_family,distance,rating
0,501,public,Pre-K,2.434192,5.333333
20880,501,public,Elementary School,0.881195,6.000000
44743,501,public,Middle School,1.731938,6.000000
68659,501,public,High School,2.963393,5.666667
95157,1001,charter,Elementary School,3.394470,4.000000
104841,1001,charter,Middle School,3.244067,5.666667
114610,1001,charter,High School,2.868138,6.000000
1,1001,public,Pre-K,3.461481,5.666667
20881,1001,public,Elementary School,1.237923,4.000000
44744,1001,public,Middle School,2.402293,3.333333


## Add Themed Ratings

In [11]:
gs_ratings_path = '../data/raw/great_schools_ratings.csv'
df_gs = pd.read_csv(gs_ratings_path)
df_gs.head()

In [17]:
df_gs1.head()

,query_zip_code,distance,type,level-codes,rating,level_family
1,14425,3.613472,public,e,5.0,NaN
2,14425,3.613472,public,m,7.0,NaN
3,14425,3.613472,public,e,3.0,NaN
4,14425,3.613472,public,h,7.0,NaN
7,14425,5.854783,public,"p,e",4.0,NaN


In [177]:
universal_id = 100012
url = f"https://gs-api.greatschools.org/schools/{universal_id:05d}/subratings"
payload={}
headers = {
  'Accept': 'application/json',
  'Content': 'application/json',
  'X-API-Key': GS_API_KEY
}

response_dist = requests.request("GET", url, headers=headers, data=payload)

gs_call_count += 1

config['great_schools_api']['call_count'] = str(gs_call_count)
with open('../SECRETS.ini', 'w') as file:
    config.write(file)

# vals = {x[0]:x[1]['value'] if 'value' in x[1] else np.nan for x in response_dist.json().items()}
# all_vals = {}
# all_vals[universal_id] = vals

In [178]:
response_dist.json()

{'status': 422, 'error': 'unprocessable_entity', 'message': 'no data found'}

In [223]:
def gs_themed_rating_io(universal_id):
#     if True:
#         return 1
    url = f"https://gs-api.greatschools.org/schools/{universal_id:05d}/subratings"
    payload={}
    headers = {
      'Accept': 'application/json',
      'Content': 'application/json',
      'X-API-Key': GS_API_KEY
    }
    try:
        response_dist = requests.request("GET", url, headers=headers, data=payload)
    except Exception as e:
        print(f'Failed extract for universal_id: {universal_id}\nSkipping....')
        return None
    
    if 'status' in response_dist.json().keys():
        return {universal_id:{}}
    
    return {universal_id:
                {x[0]:x[1]['value'] if 'value' in x[1] else np.nan for x in response_dist.json().items()}}
    

In [224]:
def extract_gs_themed_rating_chunked(chunk_ids):
    output = []
    for _id in chunk_ids:
        output.append(gs_themed_rating_io(_id))
    return output

In [225]:
def split_list(lst, n):
    """
    Function to split a list into n chunks.
    """
    return np.array_split(lst, n)

In [226]:
def extract_themed_ratings(universal_ids, cached_ratings):
    ids = sorted(set(universal_ids) - set(cached_ratings))  # Replace with your list of 100k IDs
    num_cores = 4  # Adjust this to the number of cores you want to use

    # Split the list of IDs into chunks
    chunks = split_list(ids, num_cores)

    # Use joblib to process the chunks in parallel
    results = Parallel(n_jobs=num_cores)(delayed(extract_gs_themed_rating_chunked)(chunk) for chunk in chunks)
    return results

In [227]:
def extract_themed_ratings_mt(universal_ids, cached_ratings):
    ids = sorted(set(universal_ids) - set(cached_ratings))  # Replace with your list of 100k IDs
    num_workers = 8  # Adjust this to the number of cores you want to use

    # Split the list of IDs into chunks
    chunks = split_list(ids, num_workers)

    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(extract_gs_themed_rating_chunked, chunk): chunk for chunk in chunks}
        for future in as_completed(futures):
            try:
                results.append(future.result())
            except Exception as e:
                logging.error(f"Thread execution failed: {str(e)}")
#                 results.append(None)
    return results

In [237]:
uid_rating_path = '../data/raw/great_schools_themed_ratings.csv'
all_uids = sorted(
    df_gs.dropna(subset=['rating']).sort_values(['query_zip_code','distance'])\
    .groupby(['query_zip_code','type']).head(12)['universal-id'].unique()
)

if os.path.exists(uid_rating_path):
    cached_ratings = pd.read_csv(uid_rating_path)
    cached_uids = list(cached_ratings['universal_id'].unique())
else:
    cached_uids = []
    
all_ratings = {}

results = extract_themed_ratings_mt(all_uids, cached_uids)

final_results = {}
for lst in results:
    for d in lst:
        for k, vals in d.items():
            final_results[k] = vals
            
df_themed_ratings = pd.DataFrame(final_results).T
df_themed_ratings.index.name = 'universal_id'
df_themed_ratings.reset_index(inplace=True)
if len(cached_uids) > 0:
    df_output = pd.concat([cached_ratings,df_themed_ratings]).reset_index(drop=True)
else:
    df_output = df_themed_ratings.copy()

df_output.to_csv(uid_rating_path, index=False)

In [238]:
df_output

,universal_id,test-scores-rating,college-readiness-rating,equity-rating,academic-progress-rating,student-growth-rating,attendance-flag,discipline-flag,message
0,100027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100028,9.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN
2,100019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100017,9.0,NaN,3.0,NaN,9.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
76678,4819532,10.0,NaN,4.0,NaN,9.0,NaN,NaN,NaN
76679,4819534,10.0,NaN,6.0,NaN,10.0,NaN,NaN,NaN
76680,4819544,7.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN
76681,4819547,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN


In [186]:
final_results = {}
for lst in results:
    for d in lst:
        print(d)

{100010: {}}
{100012: {}}
{100027: {}}
{100028: {'test-scores-rating': 9, 'college-readiness-rating': nan, 'equity-rating': nan, 'academic-progress-rating': nan, 'student-growth-rating': 7, 'attendance-flag': nan, 'discipline-flag': nan}}
{100013: {'test-scores-rating': nan, 'college-readiness-rating': 7, 'equity-rating': 3, 'academic-progress-rating': nan, 'student-growth-rating': nan, 'attendance-flag': nan, 'discipline-flag': nan}}
{100016: {'test-scores-rating': 9, 'college-readiness-rating': nan, 'equity-rating': 9, 'academic-progress-rating': nan, 'student-growth-rating': 10, 'attendance-flag': nan, 'discipline-flag': nan}}
{100024: {'test-scores-rating': nan, 'college-readiness-rating': 7, 'equity-rating': 4, 'academic-progress-rating': nan, 'student-growth-rating': nan, 'attendance-flag': nan, 'discipline-flag': nan}}
{100026: {}}
{100017: {'test-scores-rating': 9, 'college-readiness-rating': nan, 'equity-rating': 3, 'academic-progress-rating': nan, 'student-growth-rating': 9, 

In [217]:
pd.DataFrame(final_results).T

,test-scores-rating,college-readiness-rating,equity-rating,academic-progress-rating,student-growth-rating,attendance-flag,discipline-flag
100010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100027,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100028,9.0,NaN,NaN,NaN,7.0,NaN,NaN
100013,NaN,7.0,3.0,NaN,NaN,NaN,NaN
100016,9.0,NaN,9.0,NaN,10.0,NaN,NaN
100024,NaN,7.0,4.0,NaN,NaN,NaN,NaN
100026,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100017,9.0,NaN,3.0,NaN,9.0,NaN,NaN
100018,9.0,NaN,6.0,NaN,7.0,NaN,NaN


In [218]:
results

[[{100010: {}}, {100012: {}}],
 [{100027: {}},
  {100028: {'test-scores-rating': 9,
    'college-readiness-rating': nan,
    'equity-rating': nan,
    'academic-progress-rating': nan,
    'student-growth-rating': 7,
    'attendance-flag': nan,
    'discipline-flag': nan}}],
 [{100013: {'test-scores-rating': nan,
    'college-readiness-rating': 7,
    'equity-rating': 3,
    'academic-progress-rating': nan,
    'student-growth-rating': nan,
    'attendance-flag': nan,
    'discipline-flag': nan}},
  {100016: {'test-scores-rating': 9,
    'college-readiness-rating': nan,
    'equity-rating': 9,
    'academic-progress-rating': nan,
    'student-growth-rating': 10,
    'attendance-flag': nan,
    'discipline-flag': nan}}],
 [{100024: {'test-scores-rating': nan,
    'college-readiness-rating': 7,
    'equity-rating': 4,
    'academic-progress-rating': nan,
    'student-growth-rating': nan,
    'attendance-flag': nan,
    'discipline-flag': nan}},
  {100026: {}}],
 [{100017: {'test-scores-ra